# Analysis Review Attitude in Python

This code is main edited by Enze Wang

Fixed by Yicen Liu and Hua Tong

# Attention

Please put four .csv file in data folder and put this code in main folder instead of code folder.

This code want calculate the number of positive review and negative review on different business and different nouns. 

In [1]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
import nltk
import string
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
lmtzr = WordNetLemmatizer()
import itertools
from collections import Counter
from nltk import FreqDist
from nltk.stem import PorterStemmer
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
nltk.download('averaged_perceptron_tagger')
from wordcloud import WordCloud
from nltk import sent_tokenize, word_tokenize, PorterStemmer, pos_tag 
from nltk.corpus import stopwords
from nltk.text import TextCollection
from nltk.corpus import wordnet
import nltk.tokenize as nt
from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /Users/enze/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/enze/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/enze/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/enze/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
important_word_tf_idf=pd.read_csv("data/importance_word_tf_idf.csv")

In [3]:
pizza_business_clean_review=pd.read_csv('data/pizza_business_clean_review.csv')

In [4]:
pizza_business_review=pd.read_csv('data/pizza_review.csv')

# Review Attitude Function

# Count Positive & Negative Words

Remember we have find all positive and negative words before, therefore we can count positive and negative words in a piece of words. 

In [5]:
Positive_Words=pd.read_csv("data/Positive_Word.csv")["Word"].tolist()
Negative_Words=pd.read_csv("data/Negative_Word.csv")["Word"].tolist()

Below is our review attitude function to judge positive and negative attitude by counting positive and negative words from our list. When we scan every words, if it is positive and no Negative adverbs in the front of it, we count positive +1, if there is negative adverbs in the front of it, count negative +1, and the similar with negative. Note that there is no perfect way to judge review attitude if we do not use advanced method.

In [6]:
def Review_attitude_1(review):
    Review_word=review.split(" ")
    positive=0
    negative=0
    n=len(Review_word)
    Negative_Adverbs=["not","no","never"]
    for i in range(n):
        target=Review_word[i]
        if target in Positive_Words:
            # If there is Negative_Adverbs in front of Positive words, inverse the attitude.
            if len([word for word in Review_word[i-2:i] if word in Negative_Adverbs])>0:
                negative+=1
            else:
                positive+=1
        elif target in Negative_Words:
            if len([word for word in Review_word[i-2:i] if word in Negative_Adverbs])>0:
                positive+=1
            else:
                negative+=1
    if positive>negative:
        return(1)
    elif negative>positive:
        return(-1)
    else:
        return(0)

In [7]:
[word for word in ["not","a","good"] if word in ["not","no","never"]]

['not']

In [8]:
[word for word in ["not","no","never"] if word in Negative_Words]

[]

This three words is not in Negative_Words

When output is 1, means that function take it as positive reviews, -1 as negatives reviews, 0 as neutral reviews. 

You may asked whether the threshold is too narrow for neutral reviews. Do not worry. In the analysis we found that there is a lot of neutral reviews. The reason is that some reviews has no intersection with neither positive nor negative word list. Therefore our function will take them as neutral.

In fact the review attitude function here depends on two threshold, positive and negative word list. Therefore it is not a perfect way. 

In [9]:
Review_attitude_1("it is good")

1

In [10]:
Review_attitude_1("it is not good")

-1

In [11]:
Review_attitude_1("it is not a good apple")

-1

In [12]:
Review_attitude_1("it is not bad")

1

In [13]:
Review_attitude_1("it is not a bad apple")

1

In [14]:
Review_attitude_1("it is bad")

-1

In [15]:
Review_attitude_1("wow pizza here")

1

With the inverse attitude according to the Negative Adverb. Our function can handle not bad, not good review easily.

# Review Cut

It is never a good idea to judge a whole review:

In [16]:
example_review=pizza_business_review['text'][0].lower()
example_review

"parking is tough to get here so i don't get to go much. i also get a little cautious sometimes as they frequently have multiple questionable looking characters.\n\npro's:\n- legendary fries that used to be super huge and gave you great value for your buck\n- good burgers\n- if you drink, it looks like they have a lot of different kinds of alcohol\n\ncon's:\n- customer service? hahaha. not here. if you can get more than a fart out of one of the twin's working there, you are ahead of the pack.\n- dirty, messy tables, crap everywhere.\n- many questionable characters here. someone once asked me if i wanted to buy stolen jewelry off them while i was eating. i'm being totally honest too.\n- their french fry portions have gotten smaller. they used to need two baskets for a large fry and now only need one.\n- really expensive hot dogs. \n- they have strict limits on ketchup. no i'm not joking. 2 for large fries, 1 for anything less.\n\ni do recommend getting their legendary fries at least onc

In [17]:
Review_attitude_1(example_review)

1

As you can see, the example review at least mention three kinds of service of this business, Park, Fries and Burger. But we have no idea the positive attitude is for which kinds of service. Therefore we must cut review into pieces. This step is different from the word cut before.

In [18]:
Noun_Variables_Important=pd.read_csv('data/Noun_Variables_Important.csv')
Important_Noun=Noun_Variables_Important["0"].tolist()
Important_Noun

['pizza',
 'food',
 'cheese',
 'sauce',
 'service',
 'salad',
 'restaurant',
 'crust',
 'chicken',
 'beer',
 'menu',
 'delivery',
 'staff',
 'price',
 'bread',
 'dinner',
 'meal',
 'pasta',
 'server',
 'flavor',
 'sandwich',
 'lunch',
 'pie',
 'pepperoni',
 'location',
 'fry',
 'meatball',
 'waitress',
 'wine',
 'burger',
 'sausage',
 'garlic',
 'meat',
 'tomato',
 'appetizer',
 'manager',
 'style',
 'mushroom',
 'pepper',
 'dough',
 'dessert',
 'soup',
 'steak',
 'onion',
 'bartender',
 'bacon',
 'kitchen',
 'hoagie',
 'fan',
 'parking',
 'mozzarella',
 'employee',
 'water',
 'potato',
 'waiter',
 'entree',
 'oil',
 'cream',
 'serve',
 'shrimp',
 'fish',
 'gluten',
 'parmesan',
 'spinach',
 'beef',
 'egg',
 'brunch',
 'ice',
 'chef',
 'breadstick',
 'breakfast',
 'cake',
 'marinara',
 'pork',
 'spaghetti',
 'lettuce',
 'bbq',
 'music',
 'lasagna',
 'gelato',
 'cocktail',
 'eggplant',
 'gnocchi',
 'butter',
 'ravioli',
 'taco',
 'pesto',
 'margherita',
 'chocolate',
 'coffee',
 'takeou

In [19]:
pizza_business_review

,Unnamed: 0,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,0,hfXVxrlD6fr1kXfz6sR13Q,7nIzgNF7YzI-UAyYr3y9hw,J9f-9Prw2YVM-fiZqv2fmQ,2,0,1,1,Parking is tough to get here so I don't get to...,2013-09-04 04:35:27
1,1,EDQpLLFiAxeSxWI5ipwzBQ,SkueY7hVsyXmYItELoodGw,lpel3pRFAzlow7u2TJMz4Q,4,3,0,0,The server had an extensive knowledge of wines...,2015-04-28 19:28:12
2,2,RWmT90rP7wYo57dG1cyvww,n3ahpmnEXsZYj3nbVMoUmg,Zja7ykq3icDzkN20XX6j1w,5,1,0,1,Fast Forward\n\nEvery trip from NYC to Pitt in...,2013-07-12 23:08:35
3,3,0jduztItUhOh3FzKGjMxZQ,lVbPEaNUxFjwUydjentrYw,hr0vxeTzlHx6fTpSSmu2AA,5,1,0,0,By far my favorite Cleveland/Akron restaurant!...,2013-12-07 01:09:23
4,4,sxfmzhMvZ8LlI1x4fhNjKg,SQKRGzDa7qd8qHb13Z2-Hg,F4kjKxXXyn3e-BYVb1mWcg,4,0,0,0,All food is good here I usually get the Italia...,2014-10-27 01:42:40
...,...,...,...,...,...,...,...,...,...,...
67151,67151,gcNRaJMDlSVs8YlqwpsRuw,_Rgn4SZT2wXyQnW9AD9Tog,nR3u-wQ99bop6IdGoU38KQ,1,1,0,0,I'm originally from the other side of Ohio and...,2019-09-21 00:24:03
67152,67152,yFxDOdiDb8HPaK1io81kiw,4r33dXcE1oYZxjONrhxTiA,yT1Ad3auuw1jAJEZHr7Yqw,5,1,1,1,"Yes, yes, YES!! We loved going to the place in...",2019-09-03 04:32:16
67153,67153,Z_xmoLgIMB4As90t0a5eBQ,Iy6HkAzFcpeU7GF0e1FRSA,R13VkcyUIU3xG2fRFbBvBA,4,3,3,3,Went here for a lunch meeting amidst s typical...,2019-12-11 16:18:26
67154,67154,CzYRFCnkbqOVb3WIMMrbfw,TP9Tz7gcAfK-BeT16qpZ2w,M-caugKzJ9qLOmCBfD9Nyg,1,1,0,0,My wife and I have eaten at La Brioche at leas...,2017-06-13 02:14:53


We build a dictionary in Python, and if we scan a word in this abbreviation, it will replace with its real form. 

In [20]:
replace_words={
 "n't":"not",
 "don't":"do not",
 "aren't":"are not",
 "can't":"can not",
 "couldn't":"could not",
 "didn't":"did not",
 "doesn't":"does not",
 "hadn't":"had not",
 "hasn't":"has not",
 "haven't":"have not",
 "isn't":"is not",
 "mightn't":"might not",
 "mustn't":"must not",
 "needn't":"need not",
 "shan't":"shall not",
 "shouldn't":"should not",
 "wasn't":"was not",
 "weren't":"were not",
 "won't":"will not",
 "wouldn't":"would not"}

In [21]:
replace_words.get("don't")

'do not'

# Attention:

This process is not the same with review clean before, here we will keep "not" "no" "never" in our review.

Fix our stop_words

In [22]:
stop_words = stopwords.words('english')
stop_words = [i for i in stop_words if i not in ["not",'no','never',"n't","don't","aren't","can't","couldn't", "didn't","doesn't","hadn't","hasn't","haven't","isn't","mightn't","mustn't","needn't","shan't","shouldn't","wasn't","weren't","won't","wouldn't"]]
stop_words



['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In our code, we will replace 

! ? ; \n

with .

All sign above can split a whole review by small review.

and we split the whole review by " " then replace every "don't" "isn't" ... with "do not", "is not"...

then move every stop_words we designed. lemmatize and remove other sign. change multiple space into one space then outputs the final review by spliting with "."


For example:

In [23]:
Review="How bad here!, I will never go this place. The aren't good restaurants. It isn't a good restaurtnats. Parking is tough to get here so I don't get to go much. I also get a little cautious sometimes as they frequently have multiple questionable looking characters.\n\nPro's:\n- Legendary fries that used to be super huge and gave you great value for your buck\n- Good burgers\n- If you drink, it looks like they have a lot of different kinds of alcohol\n\nCon's:\n- Customer Service? Hahaha. Not here. If you can get more than a fart out of one of the twin's working there, you are ahead of the pack.\n- Dirty, messy tables, crap everywhere.\n- Many questionable characters here. Someone once asked me if I wanted to buy stolen jewelry off them while I was eating. I'm being totally honest too.\n- Their french fry portions have gotten smaller. They used to need two baskets for a large fry and now only need one.\n- Really expensive hot dogs. \n- They have strict limits on ketchup. No I'm not joking. 2 for large fries, 1 for anything less.\n\nI do recommend getting their legendary fries at least once and I do recommend the egg burger, but get it to go and go when it is light out."
Review

"How bad here!, I will never go this place. The aren't good restaurants. It isn't a good restaurtnats. Parking is tough to get here so I don't get to go much. I also get a little cautious sometimes as they frequently have multiple questionable looking characters.\n\nPro's:\n- Legendary fries that used to be super huge and gave you great value for your buck\n- Good burgers\n- If you drink, it looks like they have a lot of different kinds of alcohol\n\nCon's:\n- Customer Service? Hahaha. Not here. If you can get more than a fart out of one of the twin's working there, you are ahead of the pack.\n- Dirty, messy tables, crap everywhere.\n- Many questionable characters here. Someone once asked me if I wanted to buy stolen jewelry off them while I was eating. I'm being totally honest too.\n- Their french fry portions have gotten smaller. They used to need two baskets for a large fry and now only need one.\n- Really expensive hot dogs. \n- They have strict limits on ketchup. No I'm not joking

In [24]:
review=Review
review=review.lower()
Split_Review=review.replace("!",".")
Split_Review=Split_Review.replace("?",".")
Split_Review=Split_Review.replace(";",".")
Split_Review=Split_Review.replace("\n",".")
Split_Review=Split_Review.split(" ")
clean_review=[]
for i in Split_Review:
    if i in set(replace_words.keys()):
        clean_review.append(replace_words.get(i))
    else:
        clean_review.append(i)
clean_review=(" ".join(clean_review)).split(" ")
review=[i for i in clean_review if i not in stop_words]     
review=[lmtzr.lemmatize(i) for i in review]
review=" ".join(review)
review="".join([i for i in review if i not in '"#$%&\'()*+,-/:<=>@[\\]^_`{|}~\'\'``-'])
review = re.sub(' +',' ',review)
Split_Review=review.split(".")
Split_Review=[i for i in Split_Review if i not in [" ",""]]
Split_Review

['bad here',
 ' never go place',
 ' not good restaurants',
 ' not good restaurtnats',
 ' parking tough get not get go much',
 ' also get little cautious sometimes frequently multiple questionable looking characters',
 'pros',
 ' legendary fry used super huge gave great value buck',
 ' good burgers',
 ' drink look like lot different kind alcohol',
 'cons',
 ' customer service',
 ' hahaha',
 ' not here',
 ' get fart one twins working there ahead pack',
 ' dirty messy tables crap everywhere',
 ' many questionable character here',
 ' someone asked wanted buy stolen jewelry eating',
 ' im totally honest too',
 ' french fry portion gotten smaller',
 ' used need two basket large fry need one',
 ' really expensive hot dogs',
 ' strict limit ketchup',
 ' no im not joking',
 ' 2 large fries 1 anything less',
 'i recommend getting legendary fry least recommend egg burger get go go light out']

As you can see, the whole review split into several small reviews. Works well. 

There is no perfect way to split the review, most customers like use , instead of . in their review. But the whole review attitude needs more detailed study to design it

In [25]:
Clean_Review=[]
Review=pizza_business_review["text"]
for i in range(len(Review)):
    review=Review[i]
    review=review.lower()
    Split_Review=review.replace("!",".")
    Split_Review=Split_Review.replace("?",".")
    Split_Review=Split_Review.replace(";",".")
    Split_Review=Split_Review.replace("\n",".")
    Split_Review=Split_Review.split(" ")
    clean_review=[]
    for i in Split_Review:
        if i in set(replace_words.keys()):
            clean_review.append(replace_words.get(i))
        else:
            clean_review.append(i)
    clean_review=(" ".join(clean_review)).split(" ")
    review=[i for i in clean_review if i not in stop_words]     
    review=[lmtzr.lemmatize(i) for i in review]
    review=" ".join(review)
    review="".join([i for i in review if i not in '"#$%&\'()*+,-/:<=>@[\\]^_`{|}~\'\'``-'])
    review = re.sub(' +',' ',review)
    Split_Review=review.split(".")
    Split_Review=[i for i in Split_Review if i not in [" ",""]]
    Split_Review
    Clean_Review.append(Split_Review)

In [26]:
pizza_business_review["Clean_Review"]=Clean_Review
pizza_business_review.to_csv("data/Split_Review.csv")

Therefore we cut review into several piece of review, split by some important symbols.

# Attention: Code running for long time below:

In [27]:
r,c=pizza_business_review.shape
c=len(Important_Noun)
Attitude=np.zeros([r,c])
for i in range(r):
    for j in range(c):
        Target=Important_Noun[j]
        Review=pizza_business_review["Clean_Review"][i]
        Target_Review=[]
        for Small_Review in Review:
            if Target in Small_Review.split(" "):
                Target_Review.append(Small_Review)
        Target_Review=".".join(Target_Review)
        if any(c.isalpha() for c in Target_Review)==True:
            Attitude[i][j]=Review_attitude_1(Target_Review)
        else:
            Attitude[i][j]=np.nan

How the loop above works??

For example, when i=27 and j=1, our target word and review will be:

In [28]:
Target=Important_Noun[1]
Target

'food'

In [29]:
Review=pizza_business_review["Clean_Review"][27]
Review

['visiting ca arrived hotel sunday night yesterday',
 ' not lot option front desk suggested place',
 ' got shrimp sandwich jalapeño popper delivery',
 ' food took called restaurant worker sounded baffled bc order lost',
 ' not manager matt believe came line help out',
 ' guess driver gotten lost went wrong hotel',
 ' middle meal realized not get receipt called back manager willing drop way home',
 ' thought really sweet insisted drop tomorrow',
 ' still woke receipt must say really impressed him',
 ' fact im town clearly mean not regular here not go way',
 ' unfortunate food late unlike server answered phone casually said order not placed handed phone manager not want deal me manager going way make sure happy really stood me',
 ' also food really delicious']

Then our function will extract every small review include the target word "food" and combine it as a whole review. 

In [30]:

Target_Review=[]
for Small_Review in Review:
    if Target in Small_Review.split(" "):
        Target_Review.append(Small_Review)
Target_Review=".".join(Target_Review)
Target_Review

' food took called restaurant worker sounded baffled bc order lost. unfortunate food late unlike server answered phone casually said order not placed handed phone manager not want deal me manager going way make sure happy really stood me. also food really delicious'

As you can see, the customers said a lot about "food" in his review.

We check is it a real review or some space which has no means, if there is no valid review, just return np.nan

In [31]:
 any(c.isalpha() for c in Target_Review)==True

True

Then give a attitude toward this review and fill in -1,0 or 1 into our matrix:

In [32]:
Review_attitude_1(Target_Review)

1

Therefore, the i and j entry of Attitude matrix is whether negative, neutral or positive of review i towards important words j. 

In [33]:
Attitude[27][1]

1.0

In [34]:
review_attitude=pd.DataFrame(Attitude, columns=Important_Noun)
review_attitude

,pizza,food,cheese,sauce,service,salad,restaurant,crust,chicken,beer,...,cranberry,linguini,fennel,fryer,pea,coconut,mixture,peach,manicotti,romaine
0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,1.0,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67151,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67152,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67153,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67154,NaN,1.0,NaN,NaN,1.0,NaN,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
review_attitude = pd.concat([pizza_business_review["review_id"],pizza_business_review["business_id"],pizza_business_review["stars"], review_attitude], axis=1)

In [36]:
review_attitude.to_csv("data/Word_Review_Matrix.csv")
review_attitude

,review_id,business_id,stars,pizza,food,cheese,sauce,service,salad,restaurant,...,cranberry,linguini,fennel,fryer,pea,coconut,mixture,peach,manicotti,romaine
0,hfXVxrlD6fr1kXfz6sR13Q,J9f-9Prw2YVM-fiZqv2fmQ,2,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EDQpLLFiAxeSxWI5ipwzBQ,lpel3pRFAzlow7u2TJMz4Q,4,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,RWmT90rP7wYo57dG1cyvww,Zja7ykq3icDzkN20XX6j1w,5,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0jduztItUhOh3FzKGjMxZQ,hr0vxeTzlHx6fTpSSmu2AA,5,NaN,1.0,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,sxfmzhMvZ8LlI1x4fhNjKg,F4kjKxXXyn3e-BYVb1mWcg,4,1.0,1.0,NaN,-1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67151,gcNRaJMDlSVs8YlqwpsRuw,nR3u-wQ99bop6IdGoU38KQ,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67152,yFxDOdiDb8HPaK1io81kiw,yT1Ad3auuw1jAJEZHr7Yqw,5,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67153,Z_xmoLgIMB4As90t0a5eBQ,R13VkcyUIU3xG2fRFbBvBA,4,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67154,CzYRFCnkbqOVb3WIMMrbfw,M-caugKzJ9qLOmCBfD9Nyg,1,NaN,1.0,NaN,NaN,1.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


This matrix is similar with word matrix before, but here we have 4 kinds of entry 1 for positive review, -1 for negative review, "0" for neutral review and np.nan for noun review. 